# Informações de Canal do You Tube e Gráficos do Bokeh

In [ ]:
#Importa as bibliotecas 
import urllib 
from urllib.request import urlopen
import json
from csv import DictReader

In [ ]:
import pandas as pd

In [ ]:
import bokeh
from bokeh.io import show,output_notebook
from bokeh.plotting import figure, output_file
from bokeh.models import ColumnDataSource
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral6

In [ ]:
#Carrega o bokeh
output_notebook()

In [ ]:
#Impede que o bokeh abra novas abas
bokeh.io.reset_output()
bokeh.io.output_notebook()

In [ ]:
from bokeh.models import ColumnDataSource, Label, LabelSet, Range1d

In [ ]:
#Captura a chave API a partir de um arquivo csv
#O arquivo csv está salvo na mesma pasta em que esse notebook foi feito

with open('YouTubeApiKey.csv') as arquivo_csv:
    reader = DictReader(arquivo_csv)
    for linha in reader:
        chave_api = linha['youTubeApiKey']

# JAMAIS PUBLIQUE A SUA CHAVE API!!!!!!

In [ ]:
#Declara o ID do canal da TransUnion Brasil
canal_ID = 'UCApAENg_n8hM8uTQKh-LxCg'

In [ ]:
#Declara as variáveis para manipulação do canal
url = 'https://www.googleapis.com/youtube/v3/search?part=snippet&channelId='+canal_ID+'&maxResults=50&type=video&key='+chave_api
response =  urllib.request.urlopen(url)
videos = json.load(response) 

#Lista que armazena os ids dos vídeos
videoMetadata = [] 

#Repetição para inserir os vídeos na lista videoMetadata
for video in videos['items']:
    if video['id']['kind'] == 'youtube#video':
        videoMetadata.append(video['id']['videoId'])

In [ ]:
for metadata in videoMetadata:
    Video_ID = metadata
    Video_Url = 'https://www.googleapis.com/youtube/v3/videos?part=snippet%2CcontentDetails%2Cstatistics&id='
        +Video_ID+'&key='+chave_api
    response = urllib.request.urlopen(Video_Url)
    videos = json.load(response) 
    videoMetadata = [] 
    for video in videos['items']: 
        if video['kind'] == 'youtube#video':
        
            variaveis =  ['Nº de Likes','Nº de Dislikes','Nº de Comentários','Nº de Favoritos']
            contagem =   [int(video['statistics']['likeCount']),
                          int(video['statistics']['dislikeCount']),int(video['statistics']['commentCount']),
                          int(video['statistics']['favoriteCount'])]
            
            source = ColumnDataSource(data = dict(variaveis=variaveis,contagem = contagem))
            
            p = figure(x_range=variaveis,plot_height=200,toolbar_location=None,title=video['snippet']['title'])
            p.vbar(x='variaveis',top='contagem',width=0.7,source=source,line_color='white',
               fill_color=factor_cmap('variaveis',palette=Spectral6,factors=variaveis))
            
            citation = Label(x=100, y=120, x_units='screen', y_units='screen',
                 text='Nº de Visualizações do Vídeo:'+video['statistics']['viewCount'], render_mode='css',
                 border_line_color='black', border_line_alpha=0,
                 background_fill_color='white', background_fill_alpha=1.0)
        
            p.xgrid.grid_line_color = None
            p.ygrid.grid_line_color = None
            p.y_range.start =0
            p.y_range.end = 20
            p.add_layout(citation)
            
            print()
            
            show(p)